In [1]:
%load_ext cython

In [2]:
import pythran
%load_ext pythran.magic

In [3]:
import numpy as np

In [4]:
from numba import jit, njit

# spectral

## Cython

In [5]:
%%cython
# Author: Pim Schellart
# 2010 - 2011

"""Tools for spectral analysis of unequally sampled signals."""

from __future__ import absolute_import

import numpy as np
cimport numpy as np
cimport cython



cdef extern from "math.h":
    double cos(double)
    double sin(double)
    double atan2(double, double)

@cython.boundscheck(False)
def lombscargle_cython(np.ndarray[np.float64_t, ndim=1] x,
                np.ndarray[np.float64_t, ndim=1] y,
                np.ndarray[np.float64_t, ndim=1] freqs):
    """
    _lombscargle(x, y, freqs)

    Computes the Lomb-Scargle periodogram.
    
    Parameters
    ----------
    x : array_like
        Sample times.
    y : array_like
        Measurement values (must be registered so the mean is zero).
    freqs : array_like
        Angular frequencies for output periodogram.

    Returns
    -------
    pgram : array_like
        Lomb-Scargle periodogram.

    Raises
    ------
    ValueError
        If the input arrays `x` and `y` do not have the same shape.

    See also
    --------
    lombscargle

    """

    # Check input sizes
    if x.shape[0] != y.shape[0]:
        raise ValueError("Input arrays do not have the same size.")

    # Create empty array for output periodogram
    pgram = np.empty(freqs.shape[0], dtype=np.float64)

    # Local variables
    cdef Py_ssize_t i, j
    cdef double c, s, xc, xs, cc, ss, cs
    cdef double tau, c_tau, s_tau, c_tau2, s_tau2, cs_tau

    for i in range(freqs.shape[0]):

        xc = 0.
        xs = 0.
        cc = 0.
        ss = 0.
        cs = 0.

        for j in range(x.shape[0]):

            c = cos(freqs[i] * x[j])
            s = sin(freqs[i] * x[j])
            
            xc += y[j] * c
            xs += y[j] * s
            cc += c * c
            ss += s * s
            cs += c * s

        tau = atan2(2 * cs, cc - ss) / (2 * freqs[i])
        c_tau = cos(freqs[i] * tau)
        s_tau = sin(freqs[i] * tau)
        c_tau2 = c_tau * c_tau
        s_tau2 = s_tau * s_tau
        cs_tau = 2 * c_tau * s_tau

        pgram[i] = 0.5 * (((c_tau * xc + s_tau * xs)**2 / \
            (c_tau2 * cc + cs_tau * cs + s_tau2 * ss)) + \
            ((c_tau * xs - s_tau * xc)**2 / \
            (c_tau2 * ss - cs_tau * cs + s_tau2 * cc)))

    return pgram



## pythran

In [6]:
%%pythran
# runs twice as fast when -march=native -DUSE_BOOST_SIMD is on

import numpy as np
#pythran export lombscargle_pythran(float64[], float64[], float64[])


def lombscargle_pythran(x, y, freqs):
    """
    _lombscargle(x, y, freqs)

    Computes the Lomb-Scargle periodogram.
    
    Parameters
    ----------
    x : array_like
        Sample times.
    y : array_like
        Measurement values (must be registered so the mean is zero).
    freqs : array_like
        Angular frequencies for output periodogram.

    Returns
    -------
    pgram : array_like
        Lomb-Scargle periodogram.

    Raises
    ------
    ValueError
        If the input arrays `x` and `y` do not have the same shape.

    See also
    --------
    lombscargle

    """

    # Check input sizes
    if x.shape != y.shape:
        raise ValueError("Input arrays do not have the same size.")

    # Local variables
    c = np.cos(freqs[:, None] * x)
    s = np.sin(freqs[:, None] * x)
    xc = np.sum(y * c, axis=1)
    xs = np.sum(y * s, axis=1)
    cc = np.sum(c ** 2, axis=1)
    ss = np.sum(s * s, axis=1)
    cs = np.sum(c * s, axis=1)
    tau = np.arctan2(2 * cs, cc - ss) / (2 * freqs)
    c_tau = np.cos(freqs * tau)
    s_tau = np.sin(freqs * tau)
    c_tau2 = c_tau * c_tau
    s_tau2 = s_tau * s_tau
    cs_tau = 2 * c_tau * s_tau

    pgram = 0.5 * (((c_tau * xc + s_tau * xs)**2 / \
        (c_tau2 * cc + cs_tau * cs + s_tau2 * ss)) + \
        ((c_tau * xs - s_tau * xc)**2 / \
        (c_tau2 * ss - cs_tau * cs + s_tau2 * cc)))



    return pgram

## Numba

In [7]:
@njit
def lombscargle_numba(x, y, freqs):
    """
    _lombscargle(x, y, freqs)

    Computes the Lomb-Scargle periodogram.
    
    Parameters
    ----------
    x : array_like
        Sample times.
    y : array_like
        Measurement values (must be registered so the mean is zero).
    freqs : array_like
        Angular frequencies for output periodogram.

    Returns
    -------
    pgram : array_like
        Lomb-Scargle periodogram.

    Raises
    ------
    ValueError
        If the input arrays `x` and `y` do not have the same shape.

    See also
    --------
    lombscargle

    """

    # Check input sizes
    #if x.shape[0] != y.shape[0]:
    #    raise ValueError("Input arrays do not have the same size.")

    # Create empty array for output periodogram
    pgram = np.empty(freqs.shape[0], dtype=np.float64)

    # Local variables
    for i in range(freqs.shape[0]):

        xc = 0.
        xs = 0.
        cc = 0.
        ss = 0.
        cs = 0.

        for j in range(x.shape[0]):

            c = np.cos(freqs[i] * x[j])
            s = np.sin(freqs[i] * x[j])
            
            xc += y[j] * c
            xs += y[j] * s
            cc += c * c
            ss += s * s
            cs += c * s

        tau = np.arctan2(2 * cs, cc - ss) / (2 * freqs[i])
        c_tau = np.cos(freqs[i] * tau)
        s_tau = np.sin(freqs[i] * tau)
        c_tau2 = c_tau * c_tau
        s_tau2 = s_tau * s_tau
        cs_tau = 2 * c_tau * s_tau

        pgram[i] = 0.5 * (((c_tau * xc + s_tau * xs)**2 / \
            (c_tau2 * cc + cs_tau * cs + s_tau2 * ss)) + \
            ((c_tau * xs - s_tau * xc)**2 / \
            (c_tau2 * ss - cs_tau * cs + s_tau2 * cc)))

    return pgram



In [8]:
@jit
def lombscargle_numba_obj(x, y, freqs):
    """
    _lombscargle(x, y, freqs)

    Computes the Lomb-Scargle periodogram.
    
    Parameters
    ----------
    x : array_like
        Sample times.
    y : array_like
        Measurement values (must be registered so the mean is zero).
    freqs : array_like
        Angular frequencies for output periodogram.

    Returns
    -------
    pgram : array_like
        Lomb-Scargle periodogram.

    Raises
    ------
    ValueError
        If the input arrays `x` and `y` do not have the same shape.

    See also
    --------
    lombscargle

    """

    # Check input sizes
    if x.shape[0] != y.shape[0]:
        raise ValueError("Input arrays do not have the same size.")

    # Create empty array for output periodogram
    pgram = np.empty(freqs.shape[0], dtype=np.float64)

    # Local variables
    for i in range(freqs.shape[0]):

        xc = 0.
        xs = 0.
        cc = 0.
        ss = 0.
        cs = 0.

        for j in range(x.shape[0]):

            c = np.cos(freqs[i] * x[j])
            s = np.sin(freqs[i] * x[j])
            
            xc += y[j] * c
            xs += y[j] * s
            cc += c * c
            ss += s * s
            cs += c * s

        tau = np.arctan2(2 * cs, cc - ss) / (2 * freqs[i])
        c_tau = np.cos(freqs[i] * tau)
        s_tau = np.sin(freqs[i] * tau)
        c_tau2 = c_tau * c_tau
        s_tau2 = s_tau * s_tau
        cs_tau = 2 * c_tau * s_tau

        pgram[i] = 0.5 * (((c_tau * xc + s_tau * xs)**2 / \
            (c_tau2 * cc + cs_tau * cs + s_tau2 * ss)) + \
            ((c_tau * xs - s_tau * xc)**2 / \
            (c_tau2 * ss - cs_tau * cs + s_tau2 * cc)))

    return pgram



## Benchmark

In [9]:
n = 1000.
args = np.arange(2., 2. + n), np.arange(1., 1. + n), np.arange(3., 3. + n)

In [10]:
lombscargle_cython(*args)

array([  2.51369039e+02,   3.02591771e+02,   6.97772847e+02,
         1.25343511e+04,   2.03059534e+03,   4.36551168e+02,
         2.61750529e+02,   2.72237124e+02,   5.02246867e+02,
         3.19771995e+03,   5.40127033e+03,   5.79163135e+02,
         2.84283974e+02,   2.56036327e+02,   3.92336723e+02,
         1.47037851e+03,   4.43756559e+04,   8.44549169e+02,
         3.23200954e+02,   2.64358236e+02,   3.26611696e+02,
         8.67840731e+02,   5.64582167e+04,   1.41591020e+03,
         3.87073628e+02,   2.54702587e+02,   2.86626479e+02,
         5.91295537e+02,   5.84677285e+03,   3.01744416e+03,
         4.93494448e+02,   2.70646379e+02,   2.63423774e+02,
         4.43742580e+02,   2.12761095e+03,   1.11610199e+04,
         6.81763771e+02,   3.00150543e+02,   2.52782177e+02,
         3.57831183e+02,   1.12320016e+03,   3.45753363e+06,
         1.05347662e+03,   3.49260424e+02,   2.50860256e+02,
         3.05633783e+02,   7.15327870e+02,   1.41787089e+04,
         1.94104469e+03,

In [11]:
lombscargle_pythran(*args)

array([  2.51369039e+02,   3.02591771e+02,   6.97772847e+02,
         1.25343511e+04,   2.03059534e+03,   4.36551168e+02,
         2.61750529e+02,   2.72237124e+02,   5.02246867e+02,
         3.19771995e+03,   5.40127033e+03,   5.79163135e+02,
         2.84283974e+02,   2.56036327e+02,   3.92336723e+02,
         1.47037851e+03,   4.43756559e+04,   8.44549169e+02,
         3.23200954e+02,   2.64358236e+02,   3.26611696e+02,
         8.67840731e+02,   5.64582167e+04,   1.41591020e+03,
         3.87073628e+02,   2.54702587e+02,   2.86626479e+02,
         5.91295537e+02,   5.84677285e+03,   3.01744416e+03,
         4.93494448e+02,   2.70646379e+02,   2.63423774e+02,
         4.43742580e+02,   2.12761095e+03,   1.11610199e+04,
         6.81763771e+02,   3.00150543e+02,   2.52782177e+02,
         3.57831183e+02,   1.12320016e+03,   3.45753363e+06,
         1.05347662e+03,   3.49260424e+02,   2.50860256e+02,
         3.05633783e+02,   7.15327870e+02,   1.41787089e+04,
         1.94104469e+03,

In [12]:
lombscargle_numba(*args)

array([  2.51369039e+02,   3.02591771e+02,   6.97772847e+02,
         1.25343511e+04,   2.03059534e+03,   4.36551168e+02,
         2.61750529e+02,   2.72237124e+02,   5.02246867e+02,
         3.19771995e+03,   5.40127033e+03,   5.79163135e+02,
         2.84283974e+02,   2.56036327e+02,   3.92336723e+02,
         1.47037851e+03,   4.43756559e+04,   8.44549169e+02,
         3.23200954e+02,   2.64358236e+02,   3.26611696e+02,
         8.67840731e+02,   5.64582167e+04,   1.41591020e+03,
         3.87073628e+02,   2.54702587e+02,   2.86626479e+02,
         5.91295537e+02,   5.84677285e+03,   3.01744416e+03,
         4.93494448e+02,   2.70646379e+02,   2.63423774e+02,
         4.43742580e+02,   2.12761095e+03,   1.11610199e+04,
         6.81763771e+02,   3.00150543e+02,   2.52782177e+02,
         3.57831183e+02,   1.12320016e+03,   3.45753363e+06,
         1.05347662e+03,   3.49260424e+02,   2.50860256e+02,
         3.05633783e+02,   7.15327870e+02,   1.41787089e+04,
         1.94104469e+03,

In [13]:
lombscargle_numba_obj(*args)

array([  2.51369039e+02,   3.02591771e+02,   6.97772847e+02,
         1.25343511e+04,   2.03059534e+03,   4.36551168e+02,
         2.61750529e+02,   2.72237124e+02,   5.02246867e+02,
         3.19771995e+03,   5.40127033e+03,   5.79163135e+02,
         2.84283974e+02,   2.56036327e+02,   3.92336723e+02,
         1.47037851e+03,   4.43756559e+04,   8.44549169e+02,
         3.23200954e+02,   2.64358236e+02,   3.26611696e+02,
         8.67840731e+02,   5.64582167e+04,   1.41591020e+03,
         3.87073628e+02,   2.54702587e+02,   2.86626479e+02,
         5.91295537e+02,   5.84677285e+03,   3.01744416e+03,
         4.93494448e+02,   2.70646379e+02,   2.63423774e+02,
         4.43742580e+02,   2.12761095e+03,   1.11610199e+04,
         6.81763771e+02,   3.00150543e+02,   2.52782177e+02,
         3.57831183e+02,   1.12320016e+03,   3.45753363e+06,
         1.05347662e+03,   3.49260424e+02,   2.50860256e+02,
         3.05633783e+02,   7.15327870e+02,   1.41787089e+04,
         1.94104469e+03,

In [14]:
%timeit lombscargle_cython(*args)

49.7 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit lombscargle_pythran(*args)

59.5 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit lombscargle_numba(*args)

48.9 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit lombscargle_numba_obj(*args)

48.9 ms ± 635 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
